In [3]:
from datasets import load_dataset

data = load_dataset("nkarthik1912/SA")

In [40]:
data["train"][0]['label']

'neutral'

In [2]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

2024-04-07 09:45:38.528310: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 09:45:38.565192: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 09:45:38.565216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 09:45:38.566123: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 09:45:38.572171: I tensorflow/core/platform/cpu_feature_guar

In [10]:
from huggingface_hub import notebook_login
notebook_login()

from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

dataset_name = "nkarthik1912/SA"
dataset = load_dataset(dataset_name, split="train")

base_model_name = "NousResearch/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)
base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

output_dir = "./results"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=500
)

max_seq_length = 512

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

trainer.train()

import os
output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pavansai-vizag (fantastic4). Use `wandb login --relogin` to force relogin


Step,Training Loss
10,4.792300
20,4.354400
30,4.221600
40,4.155700
50,3.985600
60,4.054900
70,4.110600
80,3.996700
90,4.004800
100,3.939000


In [10]:
output_dir = "./results"
import os
output_dir = os.path.join(output_dir, "final_checkpoint")
device_map = {"": 0}
base_model_name = "NousResearch/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token



In [27]:
from transformers import AutoTokenizer

model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map=device_map, torch_dtype=torch.bfloat16)

inputs = tokenizer(text, return_tensors="pt").to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:

with torch.no_grad():
    logits = model(**inputs).logits

tensor([[[-13.0625,  -4.2812,  -0.9062,  ...,  -6.2812,  -7.9688,  -7.1562],
         [-10.4375, -10.6875,  -4.0938,  ...,  -7.3125,  -6.9062,  -4.5938],
         [-11.1875, -10.5625,  -0.9336,  ...,  -7.6875,  -4.7188,  -5.1250],
         ...,
         [ -8.0625,  -8.9375,   0.0488,  ...,  -3.8281,  -5.8125,  -4.5938],
         [ -8.4375,  -9.6875,   5.5938,  ...,  -3.3125,  -7.5625,  -3.9844],
         [ -5.5625,   0.5703,  14.1250,  ...,  -1.8281,  -4.3438,  -0.6328]]],
       device='cuda:0')

In [22]:
from peft import AutoPeftModelForCausalLM
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map=device_map, torch_dtype=torch.bfloat16)
text = "RahulGandhi television media congress ke liye nhi h Ye toh aapko pata chal hi gya hoga Achha hoga ki Congress ke https t co HmH8M7PTaK"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model(**inputs)
outputs[0]
# predicted_probabilities = torch.softmax(outputs.logits, dim=1).tolist()[0]
# predicted_sentiment_class = predicted_probabilities.index(max(predicted_probabilities))
# sentiment_labels = ["negative", "neutral", "positive"]
# predicted_sentiment_label = sentiment_labels[predicted_sentiment_class]

# print("Predicted sentiment:", predicted_sentiment_label)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tensor([[[-13.0625,  -4.2812,  -0.9062,  ...,  -6.2812,  -7.9688,  -7.1562],
         [-10.4375, -10.6875,  -4.0938,  ...,  -7.3125,  -6.9062,  -4.5938],
         [-11.1875, -10.5625,  -0.9336,  ...,  -7.6875,  -4.7188,  -5.1250],
         ...,
         [ -8.0625,  -8.9375,   0.0488,  ...,  -3.8281,  -5.8125,  -4.5938],
         [ -8.4375,  -9.6875,   5.5938,  ...,  -3.3125,  -7.5625,  -3.9844],
         [ -5.5625,   0.5703,  14.1250,  ...,  -1.8281,  -4.3438,  -0.6328]]],
       device='cuda:0')

In [5]:
!pip install -q huggingface_hub
!pip install -q -U trl transformers accelerate peft
!pip install -q -U datasets bitsandbytes einops wandb

In [6]:
!pip install -q -U datasets bitsandbytes einops wandb

In [7]:
 !pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-m5513csx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-m5513csx
  Resolved https://github.com/huggingface/peft.git to commit 26726bf1ddee6ca75ed4e1bfd292094526707a78
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-f2fjdxw9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-f2fjdxw9
  Resolved https://github.com/huggingface/transformers.git to commit 76fa17c1663a0efeca7208c20579833365584889
  Installing build dependencies ... done

In [21]:
id2label = {0: "negative",1: "neutral", 2: "positive"}
label2id = {'negative': 0, 'neutral': 1,'positive' :2}

In [55]:

    for j in data["train"]:
           j['label'] = label2id[j['label']]
        

In [60]:
data["train"][0]

{'text': 'nen vist bolest vztek smutek zmatek osam lost beznad j a nakonec jen klid Asi takhle vypad m j life',
 'label': 1}

In [51]:
label2id['neutral']

1

In [62]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [63]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [64]:
tokenized_data = data.map(preprocess_function, batched=True)

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [69]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [76]:

def acc_and_f1(preds, labels):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds, average='weighted')
    precision = precision_score(
        y_true=labels, y_pred=preds, average='weighted')
    recall = recall_score(y_true=labels, y_pred=preds, average='weighted')
    return{
        "acc": acc,
        "f1": f1,
        "acc_and_f1": (acc + f1) / 2,
        "precision": precision,
        "recall": recall
    }


In [75]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    result = acc_and_f1(preds, out_label_ids)
    return result 


In [85]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments,Trainer
from peft import LoraConfig
from trl import SFTTrainer
model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-hf", num_labels=3, id2label=id2label, label2id=label2id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuratio

In [87]:
base_model_name = "NousResearch/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)
base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

output_dir = "model"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=500,
  
)

max_seq_length = 512

trainer = SFTTrainer(
    model=base_model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    args=training_args,
)

trainer.train()

import os
output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuratio

Step,Training Loss
10,4.791900
20,4.345600
30,4.215800
40,4.150500
50,3.982000
60,4.053000
70,4.108600
80,3.995200
90,4.003100
100,3.937900


In [117]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map=device_map, torch_dtype=torch.bfloat16)
text = "byee"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model(**inputs)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [118]:
tmp_eval_loss, logits = outputs[:2]

In [1]:
from datasets import load_dataset

data = load_dataset("nkarthik1912/SA")

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = data.map(tokenize_function, batched=True)

In [3]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

[2024-04-08 05:23:43,792] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2024-04-08 05:23:44.333335: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-08 05:23:44.370335: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 05:23:44.370362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 05:23:44.371281: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 05:23:44.377320: I tensorflow/core/platform/cpu_feature_guar

In [4]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-large-uncased', 
    num_labels=3
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [6]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",per_device_train_batch_size=8,
                                 num_train_epochs=5)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.090400,1.097074,0.357000
2,1.066700,1.054760,0.406333
3,1.054800,1.038196,0.439000
4,1.045800,1.033829,0.437667
5,1.040400,1.030855,0.441000


TrainOutput(global_step=8750, training_loss=1.0651372035435267, metrics={'train_runtime': 4480.0565, 'train_samples_per_second': 15.625, 'train_steps_per_second': 1.953, 'total_flos': 6.525721638912e+16, 'train_loss': 1.0651372035435267, 'epoch': 5.0})

In [22]:
import torch
text_list = ["a very bad movie .", "resourceful and ingenious entertainment .", "it 's just incredibly dull .", "the movie 's biggest offense is its complete and utter lack of tension .",
             "impresses you with its open-endedness and surprises .", "unless you are in dire need of a diesel fix , there is no real reason to see it ."]
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)
    print(id2label[int(predictions)])

positive
positive
positive
positive
positive
positive


In [23]:
with open('hineng/test.txt',encoding='UTF-8') as rf:
    lines = rf.readlines()
y_true=[]
y_pred = []

In [24]:
for j in lines:
    data = j.split('\t')
    lab = data[1].split()
    y_true.append(lab[0])
    inputs = tokenizer.encode(data[0], return_tensors="pt").to(device)
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)
    y_pred.append(id2label[int(predictions)])

In [25]:
len(y_true)

3000

In [26]:
len(y_pred)

3000

In [27]:
from sklearn.metrics import f1_score

weighted_f1 = f1_score(y_true, y_pred, average='weighted')

In [28]:
weighted_f1

0.455442737684001